# Import Libraries
>Import necessary libraries for data manipulation and modeling.

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


# Load the Dataset
>Load the dataset from the CSV file 'products.csv'. Provide information about the data source and format.

This dataset contains information about users and the products they have purchased. The fields included are:

- **usuario_id**: A unique identifier for each user.
- **correo_producto**: The email address of the user.
- **nombre_producto**: The name of the purchased product.
- **categoria**: The category of the product.
- **descripcion**: A description of the product.


In [ ]:
# Load the dataset
data = pd.read_csv('products.csv')

# User-Product Interaction Matrix and User Similarity
>In this section, we will create a user-product interaction matrix and compute the cosine similarity between users.

In [32]:
# Create a user-product interaction matrix
# This matrix will have users as rows, products as columns, and 1s where a user has purchased a product
user_product_matrix = pd.pivot_table(data, index='usuario_id', columns='nombre_producto', aggfunc='size', fill_value=0)

# Calculate the cosine similarity matrix between users
user_similarity_matrix = cosine_similarity(user_product_matrix)

# Convert the cosine similarity matrix into a DataFrame for better readability
user_similarity_df = pd.DataFrame(user_similarity_matrix, index=user_product_matrix.index, columns=user_product_matrix.index)


# User-Based Product Recommendations
>In this section, we define a function to generate product recommendations for a given user based on user similarity. The function will recommend products that similar users have purchased but the current user has not.


In [38]:
def get_user_recommendations(user_id, train_data, user_similarity_df, num_recommendations=5):
    """
    Generate product recommendations for a given user based on user similarity.

    Parameters:
    user_id (String): The ID of the user for whom recommendations are to be generated.
    train_data (DataFrame): The user-product interaction matrix.
    user_similarity_df (DataFrame): The user similarity matrix.
    num_recommendations (int): The number of recommendations to generate.

    Returns:
    list: A list of recommended product names.
    """
    # Check if the user is in the training data
    if user_id not in train_data.index:
        print(f"User {user_id} has no purchase history.")
        top_products = train_data.sum().nlargest(num_recommendations).index
        return list(top_products)

    # Get the similarity row for the user
    user_similarity_row = user_similarity_df.loc[user_id]

    # Get similar users sorted by similarity, excluding the current user
    similar_users = user_similarity_row.drop(user_id).sort_values(ascending=False)

    # Generate recommendations based on similar users
    recommendations = []
    user_purchased_products = set(train_data.loc[user_id][train_data.loc[user_id] > 0].index)

    for similar_user_id in similar_users.index:
        similar_user_history = train_data.loc[similar_user_id]
        similar_user_purchased_products = set(similar_user_history[similar_user_history > 0].index)

        # Recommend products that similar users have purchased but the current user has not
        recommended_products = similar_user_purchased_products - user_purchased_products
        recommendations.extend(recommended_products)

        # Limit the number of recommendations
        if len(recommendations) >= num_recommendations:
            break

    # Remove duplicates and limit the number of recommendations
    recommendations = list(set(recommendations))
    if len(recommendations) < num_recommendations:
        remaining_recommendations = num_recommendations - len(recommendations)
        top_products = train_data.sum().nlargest(remaining_recommendations).index
        recommendations.extend(top_products)

    recommendations = recommendations[:num_recommendations]  # Limit the total number of recommendations

    return recommendations

# Example Usage of the Recommendation Function

>In this section, we will demonstrate how to use the `get_user_recommendations` function to generate product recommendations for an existing and not existing user.

In [43]:
# Example usage of the get_user_recommendations function
user_id = 'usuario3'  # Define the user ID for which recommendations are to be generated

# Generate product recommendations for the specified user
recommended_products = get_user_recommendations(user_id, user_product_matrix, user_similarity_df, num_recommendations=5)

# Print the recommended products
print(f"Recommended products for {user_id}: {recommended_products}")


Recommended products for usuario4: ['Galletas Emperador Gamesa', 'Sabritas Original', 'Churrumais Barcel', 'Triki Trakes Gamesa', 'Galletas Marías Gamesa']


In [ ]:
# Example of using the get_user recommendations function for a user with no purchase history
user_id = 'usuario343'  # Define the user ID for which recommendations are to be generated

# Generate product recommendations for the specified user
recommended_products = get_user_recommendations(user_id, user_product_matrix, user_similarity_df, num_recommendations=5)

# Print the recommended products
print(f"Recommended products for {user_id}: {recommended_products}")